In [4]:
# import
import importlib
import pandas as pd
from pipeline_executor import PipelineExecutor
import nlp_analysis.word_wizard as ww

# Supress warnings
import warnings
warnings.filterwarnings('ignore')

2023-06-22 09:19:42.883069: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-

In [8]:
importlib.reload(ww)

<module 'nlp_analysis.word_wizard' from '/Users/florian/Documents/Learning/MiBA/classes/term3/Capstone/MIBA-2023-CAPSTONE-RB-NLP/nlp_analysis/word_wizard.py'>

In [81]:
# get data
topic = "'Edge Computing' AND 'Research'"
topic2 = "Roland Berger"
pipeline_executor = PipelineExecutor()
edge = pipeline_executor.execute(query=topic, max_articles=None, overwrite=False)

Getting news article info: 100%|██████████| 353/353 [04:13<00:00,  1.39it/s]


In [104]:
# Paragraphs - Word - silhouette
wizard = ww.WordWizard(df=edge, interest = 'paragraph')
wizard.create_sentence_embeddings() \
.cluster_embeddings() \
.entitiy_recognition() \
.summarize_medoids() \
.find_sentiment(lean=False, device="cpu") \
.topic_modelling() \
.reduce_demensionality()

# safe wizard.df to parquet


Batches:   0%|          | 0/89 [00:03<?, ?it/s]


KeyboardInterrupt: 

In [103]:
# get unique values from wizard.df[['topics']]
wizard.df[['topics']].unique()

TypeError: unhashable type: 'list'

In [73]:
wizard.df['x'] = wizard.df.iloc[:,-1].apply(lambda x: x[0])
wizard.df['y'] = wizard.df.iloc[:,-2].apply(lambda x: x[1])

# count cluster size
wizard.df['cluster_size'] = wizard.df.groupby('paragraph_sentence_embeddings_clusters')['paragraph_sentence_embeddings_clusters'].transform('count')

# aggregate cluster sentiment
wizard.df['cluster_sentiment'] = wizard.df.groupby('paragraph_sentence_embeddings_clusters')['paragraph_sentiment'].transform('mean')

# keep only medoids
wizard.df = wizard.df[wizard.df['paragraph_sentence_embeddings_clusters_medoids'] == True]

# keep only cluster_size, x, y, cluster_sentiment, paragraph_clusters_sentence_embeddings_NER, topics
ner_col = [col for col in wizard.df.columns if col.endswith('NER')][0]
wizard.df = wizard.df[['cluster_size', 'x', 'y', 'cluster_sentiment', ner_col, 'topics']]

# rename columns
wizard.df.columns = ['size', 'x', 'y', 'sentiment', 'entities', 'topics']

In [78]:
wizard.df.columns = ['size', 'x', 'y', 'sentiment', 'entities', 'topics']
wizard.df.to_csv('data/RB.csv', index=False)

In [6]:
# Paragraphs - Sentence - silhouette
wizard2 = ww.WordWizard(df=quantum, interest = 'paragraph')
wizard2.create_word_embeddings() \
    .cluster_embeddings() \
    .entitiy_recognition() \
    .summarize_medoids() \
    .find_sentiment(lean=False, device="cpu") \
    .topic_modelling()

Batches:   0%|          | 0/104 [00:00<?, ?it/s]

Extracting organizations from paragraph:   0%|          | 0/5 [00:00<?, ?it/s]

Creating summaries for cluster medoids based on paragraph:   0%|          | 0/10 [00:00<?, ?it/s]

Calculating sentiment for paragraph:   0%|          | 0/3056 [00:00<?, ?it/s]

In [7]:
# Article - Word - silhouette
wizard3 = ww.WordWizard(df=quantum, interest = 'body')
wizard3.create_word_embeddings() \
    .cluster_embeddings() \
    .entitiy_recognition() \
    .summarize_medoids() \
    .find_sentiment(lean=False, device="cpu") \
    .topic_modelling()

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Creating word embeddings for body:   0%|          | 0/304 [00:00<?, ?it/s]

Extracting organizations from body:   0%|          | 0/5 [00:00<?, ?it/s]

Creating summaries for cluster medoids based on body:   0%|          | 0/10 [00:00<?, ?it/s]

Calculating sentiment for body:   0%|          | 0/300 [00:00<?, ?it/s]

In [8]:
# Article - Sentence - silhouette
wizard4 = ww.WordWizard(df=quantum, interest = 'body')
wizard4.create_sentence_embeddings() \
    .cluster_embeddings() \
    .entitiy_recognition() \
    .summarize_medoids() \
    .find_sentiment(lean=False, device="cpu") \
    .topic_modelling()

Batches:   0%|          | 0/10 [00:00<?, ?it/s]

Extracting organizations from body:   0%|          | 0/5 [00:00<?, ?it/s]

Creating summaries for cluster medoids based on body:   0%|          | 0/10 [00:00<?, ?it/s]

Calculating sentiment for body:   0%|          | 0/300 [00:00<?, ?it/s]

In [9]:
wizard.df.to_csv("data/para_word_sil.csv", index=False)
wizard2.df.to_csv("data/para_sent_sil.csv", index=False)
wizard3.df.to_csv("data/art_word_sil.csv", index=False)
wizard4.df.to_csv("data/art_sent_sil.csv", index=False)

# Gaining Insights about Quantum Computing from News NLP

In [53]:
para_word_sil = pd.read_csv("data/para_word_sil.csv")
para_sent_sil = pd.read_csv("data/para_sent_sil.csv")
art_word_sil = pd.read_csv("data/art_word_sil.csv")
art_sent_sil = pd.read_csv("data/art_sent_sil.csv")

In [64]:
import ast
def summarize(df, interest):
    wizard = ww.WordWizard(df=df, interest = interest)

    if 'paragraph_sentence_embeddings' in wizard.df.columns:
        embed = 'paragraph_sentence_embeddings'
    elif 'paragraph_word_embeddings' in wizard.df.columns:
        embed = 'paragraph_word_embeddings'
    elif 'body_sentence_embeddings' in wizard.df.columns:
        embed = 'body_sentence_embeddings'
    elif 'body_word_embeddings' in wizard.df.columns:
        embed = 'body_word_embeddings'

    # str to list
    try:
        wizard.df[embed] = wizard.df[embed].apply(ast.literal_eval)
        
        # reduce demensionality
        wizard.reduce_demensionality()

        # put x and y in different columns
        wizard.df['x'] = art_sent_sil.iloc[:,-1].apply(lambda x: x[0])
        wizard.df['y'] = art_sent_sil.iloc[:,-1].apply(lambda x: x[1])
        
        # count cluster size
        wizard.df['cluster_size'] = wizard.df.groupby(embed + '_clusters')[embed + '_clusters'].transform('count')
        
        # aggregate cluster sentiment
        wizard.df['cluster_sentiment'] = wizard.df.groupby(embed + '_clusters')['paragraph_sentiment'].transform('mean')
        
        # keep only medoids
        wizard.df = wizard.df[wizard.df[embed + '_clusters' + '_medoids'] == True]
        
        # keep only cluster_size, x, y, cluster_sentiment, paragraph_clusters_sentence_embeddings_NER, topics
        ner_col = [col for col in wizard.df.columns if col.endswith('NER')][0]
        wizard.df = wizard.df[['cluster_size', 'x', 'y', 'cluster_sentiment', ner_col, 'topics']]
        
        # rename columns
        wizard.df.columns = ['size', 'x', 'y', 'sentiment', 'entities', 'topics']

    except:
        # count cluster size
        wizard.df['cluster_size'] = wizard.df.groupby(embed + '_clusters')[embed + '_clusters'].transform('count')
        
        # aggregate cluster sentiment
        wizard.df['cluster_sentiment'] = wizard.df.groupby(embed + '_clusters')['paragraph_sentiment'].transform('mean')
        
        # keep only medoids
        wizard.df = wizard.df[wizard.df[embed + '_clusters' + '_medoids'] == True]
        
        # keep only cluster_size, cluster_sentiment, paragraph_clusters_sentence_embeddings_NER, topics
        ner_col = [col for col in wizard.df.columns if col.endswith('NER')][0]
        wizard.df = wizard.df[['cluster_size', 'cluster_sentiment', ner_col, 'topics']]
        
        # rename columns
        wizard.df.columns = ['size', 'sentiment', 'entities', 'topics']

    
    return wizard.df

In [65]:
for df in para_word_sil, para_sent_sil:
    df = summarize(df, interest = 'paragraph')

for df in art_word_sil, art_sent_sil:
    df = summarize(df, interest = 'body')

KeyError: "['article_index', 'paragraph'] not found in axis"